## Semantic Segmentation

## 1. Import Libraries

In [1]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import time

#%tensorflow_version 1.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## 2. Import helper methods
These methods help us perform the following tasks:
* Load the latest version of the pretrained DeepLab model
* Load the colormap from the PASCAL VOC dataset
* Adds colors to various labels, such as "pink" for people, "green" for bicycle and more
* Visualize an image, and add an overlay of colors on various regions

In [3]:
class DeepLabModel(object):
    """Class to load deeplab model and run inference."""

    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()

        graph_def = None
        # Extract frozen graph from tar archive.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.GraphDef.FromString(file_handle.read())
                break

        tar_file.close()

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.Session(graph=self.graph)

    def run(self, image):
        """Runs inference on a single image.

        Args:
          image: A PIL.Image object, raw input image.

        Returns:
          resized_image: RGB image resized from original input image.
          seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        return resized_image, seg_map


def create_cityscapes_label_colormap():
    """Creates a label colormap used in CITYSCAPES segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
    colormap = np.zeros((256, 3), dtype=np.uint8)
    colormap[0] = [128, 64, 128]
    colormap[1] = [244, 35, 232]
    colormap[2] = [70, 70, 70]
    colormap[3] = [102, 102, 156]
    colormap[4] = [190, 153, 153]
    colormap[5] = [153, 153, 153]
    colormap[6] = [250, 170, 30]
    colormap[7] = [220, 220, 0]
    colormap[8] = [107, 142, 35]
    colormap[9] = [152, 251, 152]
    colormap[10] = [70, 130, 180]
    colormap[11] = [220, 20, 60]
    colormap[12] = [255, 0, 0]
    colormap[13] = [0, 0, 142]
    colormap[14] = [0, 0, 70]
    colormap[15] = [0, 60, 100]
    colormap[16] = [0, 80, 100]
    colormap[17] = [0, 0, 230]
    colormap[18] = [119, 11, 32]

    return colormap


def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_cityscapes_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    
    save_path = img_loc.split('/')[-1]
    plt.savefig('semantic segmentation model/results/' + save_path)
    #plt.show()
    plt.close()



LABEL_NAMES = np.asarray([
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck',
    'bus', 'train', 'motorcycle', 'bicycle'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

## 3. Select a pretrained model
We have trained the DeepLab model using various backbone networks. Select one from the MODEL_NAME list.

In [4]:
model_path = 'semantic segmentation model/deeplab_cityscapes_xception71_trainfine_2018_09_08.tar.gz'
MODEL = DeepLabModel(model_path)
MODEL

## 4. Run on sample images

Select one of sample images (leave `IMAGE_URL` empty) or feed any internet image
url for inference.

Note that this colab uses single scale inference for fast computation,
so the results may slightly differ from the visualizations in the
[README](https://github.com/tensorflow/models/blob/master/research/deeplab/README.md) file,
which uses multi-scale and left-right flipped inputs.

In [5]:
#image_loc = 'data/Latest_SCOOT_GSV/55.848209730275_-4.255344777421482_180.jpg'


def run_visualization(url):
    """Inferences DeepLab model and visualizes result."""
    try:
        original_im = Image.open(url)
    except IOError:
        print('Cannot retrieve image. Please check loc: ' + url)
        return

    print('running deeplab on image %s...' % url)
    resized_im, seg_map = MODEL.run(original_im)

    """Get percentage of each object in seg_map"""
    labels, counts = np.unique(seg_map, return_counts=True)
    percentage = counts / counts.sum()
    objects = LABEL_NAMES[labels]
    object_percentage = pd.DataFrame({'object': objects, 'percentage': percentage}, columns=['object', 'percentage'])
    object_percentage_t = object_percentage.T
    
    object_percentage_t.columns = object_percentage_t.loc['object']
    object_percentage_t = object_percentage_t.drop(['object'])
    object_percentage_t
    #display(object_percentage_t)

    images = vis_segmentation(resized_im, seg_map)

    return object_percentage_t

## Prepare the GSV

## 1. GSV download

In [4]:
# download gsv
def get_gsv(lat, lon, heading, key, width=600, height=400):
    # heading: north: 0, east: 90, south: 180, west: 270
    url = "https://maps.googleapis.com/maps/api/streetview?size={}x{}&location={},{}&heading={}&key={}".format(width, height, lat, lon, heading, key)
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

In [15]:
import pandas as pd

# if you download the raw traffic flow data via Traffic_flow_Download.ipynb and pre-process the data via Traffic_flow_Preprocessing.ipynb
SCOOT_df = pd.read_csv('../data/final_regression_csv/detector_530.csv')
SCOOT_df_coords = SCOOT_df.loc[:,['siteId', 'lat', 'lon']]

# elif you download the pre-processed traffic flow data at: Urban Big Data Centre (UBDC) and https://zenodo.org/records/12100278.
SCOOT_df = pd.read_csv('../data/traffic_flow_data/locations.csv')
SCOOT_df_coords = SCOOT_df.loc[:,['id', 'latitude', 'longitude']]
SCOOT_df_coords = SCOOT_df_coords.rename(columns={'id': 'siteId', 'latitude': 'lat', 'longitude': 'lon'})

GSV_list = []
for i in range(len(SCOOT_df_coords)):
    lat = SCOOT_df_coords.loc[i]['lat']
    lon = SCOOT_df_coords.loc[i]['lon']
    for j in ['0', '90', '180', '270']:
        view = get_gsv(lat, lon, j, key) # get your unique GSV download code https://developers.google.com/maps/documentation/streetview/overview
        view.save('data/' + lat + '_' + lon + '_' + j + '.jpg')
        
        GSV_download = SCOOT_df_coords.loc[[i]]
        GSV_download['path'] = 'data/' + str(lat) + '_' + str(lon) + '_' + str(j) + '.jpg'
        GSV_list.append(GSV_download)

GSV_download_df = pd.concat(GSV_list)
GSV_download_df = GSV_download_df.reset_index(drop=True)

## 2. Construct the dataframe

In [12]:
GSV_download_df[['road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
                 'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck',
                 'bus', 'train', 'motorcycle', 'bicycle']] = pd.DataFrame([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], 
                                                                          index=GSV_download_df.index)

## 3. Implement the semantic segmentation on GSV

In [ ]:
for i in range(len(file_for_segmentation)):
    sum_row = file_for_segmentation.iloc[i:i+1, 4:].sum(axis=1).item()
    if sum_row == 0:
        img_loc = file_for_segmentation.path[i]
        segmentation_output = run_visualization(img_loc)
        for j in list(segmentation_output):
            file_for_segmentation.at[i, j] = segmentation_output[j].item()
        
        file_for_segmentation.to_csv('semantic segmentation model/latest_segmentation_output_before_2021-12-31.csv', index = False)

## 4. Merge the four perspective into one

In [63]:
def normalisation_GSV(data):

    site_list = list(set(data.siteId))
    objects = list(data)[4:]
    #print(len(date_list))

    site_object = {}
    for i in site_list:
        object_value = []
        for j in objects:
            b = data.loc[data.siteId == i, j].sum()/4
            object_value.append(b)
        site_object[i] = object_value

    each_site_object = {}
    site = []
    
    road = []
    sidewalk = []
    building = []
    wall = []
    fence = []
    
    pole = []
    traffic_light = []
    traffic_sign = []
    vegetation = []
    terrain = []
    
    sky = []
    person = []
    rider = []
    car = []
    truck = []
    
    bus = []
    train = []
    motorcycle = []
    bicycle = []
    
    for key, value in site_object.items():
        site.append(key)
        
        road.append(value[0])
        sidewalk.append(value[1])
        building.append(value[2])
        wall.append(value[3])
        fence.append(value[4])

        pole.append(value[5])
        traffic_light.append(value[6])
        traffic_sign.append(value[7])
        vegetation.append(value[8])
        terrain.append(value[9])

        sky.append(value[10])
        person.append(value[11])
        rider.append(value[12])
        car.append(value[13])
        truck.append(value[14])

        bus.append(value[15])
        train.append(value[16])
        motorcycle.append(value[17])
        bicycle.append(value[18])
        

    each_site_object['siteId'] = site
    each_site_object['road'] = road
    each_site_object['sidewalk'] = sidewalk
    each_site_object['building'] = building
    each_site_object['wall'] = wall
    each_site_object['fence'] = fence
    
    each_site_object['pole'] = pole
    each_site_object['traffic light'] = traffic_light
    each_site_object['traffic sign'] = traffic_sign
    each_site_object['vegetation'] = vegetation
    each_site_object['terrain'] = terrain
    
    each_site_object['sky'] = sky
    each_site_object['person'] = person
    each_site_object['rider'] = rider
    each_site_object['car'] = car
    each_site_object['truck'] = truck
    
    each_site_object['bus'] = bus
    each_site_object['train'] = train
    each_site_object['motorcycle'] = motorcycle
    each_site_object['bicycle'] = bicycle
    
    df_each_site_object = pd.DataFrame.from_dict(each_site_object)
    
    df_each_site_object = df_each_site_object.sort_values(by=['siteId'])
    df_GSV = df_each_site_object.reset_index(drop=True)
    
    return df_GSV

In [65]:
import os
new_GSV = normalisation_GSV(GSV)
save_path = '../data/independent_variables/GSV/'
os.makedirs(save_path, exist_ok=True)
new_GSV.to_csv(save_path + 'segmentation_normalisation_output.csv', index = False)